### **Data Load Proccess**:
- Objective: This notebook presents the process of loading clean data resulting from the previous phase of the project. It is intended to use a new database due to the limitations of the free [Render database](https://github.com/DCajiao/workshop001_candidates_analysis/blob/main/docs/database/how_to_deploy_databases_on_render.md) instances.
---

#### **First Step**: Write the queries for the PostgreSQL database based on the data.

---

#### **Second Step**: Upload data to database

---

#### **Results**:

---